In [232]:

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

### Impoting libraries 

In [233]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame as show
import seaborn as sns
import squarify
%matplotlib inline

### Reading and making dataframe of the csv formated file
Here we converted categorical data to 'category' type


csv file-(startup_funding.csv)- contains the information of all startups(india) from 2015 to 2017


In [234]:
df=pd.read_csv('../input/startup_funding.csv',dtype={
    
    'InvestmentType':'category'
    
})#read_csv function is used to import dataset 'startup_funding.csv'



## sample of our dataset (startup_funding.csv)



In [235]:
df.head(4)#display first 4 rows of dataframe

In [236]:
df.tail(4)#display first 4 rows of dataframe

### metadata


columns and index information

In [237]:
df.info(null_counts=True)

dimentions of dataframe


In [238]:
print('Rows {rows} and Columns {columns}'.format(rows=df.shape[0],columns=df.shape[1]))

# Cleaning Data

### Problem 

Dataset dosen't have proper format


In [239]:
df.dtypes

## repersentation of missing data

- here yellow bars repersent the null values(missing values)
- since we can see that 'remarks' has the higher density of missing data,we can neglect this column
- here we can use estimated statistical values of available data of attribute('AmountInUSD') to fill the missing values



## dataframe contains lots of NaN(null values)

In [240]:
df.apply(lambda X:sum(X.isnull()))

In [241]:

plt.figure(figsize=(10,5))
sns.heatmap(df.isnull(),cmap='viridis',yticklabels=False,cbar=False)

### specifying format error

some dates are not formated where generalized format is 'dd/mm/yy'


In [242]:
df[df['Date']=='12/05.2015']['Date']#here '.' is there instead of '/'


## Amount in usd has ','(notnumeric) chracters and null values
datatype is not numerical

In [243]:
df['AmountInUSD'].head(5)

# solving problem with missing data
- WE CAN FILL THE MISSING NUMERICAL VALUES USING FOLLWING STATISTICAL METHODS-
    -BY MODE OF CENTRAL TENDENCY
    
        - BACKWARD FILLING
    
        -FORWARD FILLING
    
        -INTERPOLATION(LINEAR)
    

## Cleaning missing data and formatting


### handeling cities null in column

In [244]:
df['CityLocation']=df['CityLocation'].fillna(value='NotSpecific')

In [245]:
df['IndustryVertical']=df['IndustryVertical'].fillna(value='Other')

### Formatting citylocation 

In [246]:
import re#importing regular expressions
def convert_Slash(x):#converts citylocation where multiple citiescentres
    x=x.lower()
    if   re.search('/',x):
        return x.split('/')[0].strip()
    else :
        return x.strip()
df['CityLocation']=df['CityLocation'].apply(convert_Slash)

## Backup of dataframe
deleting unwanted columns

In [247]:
newdf=df.copy()
del newdf['Remarks']
del newdf['SNo']


### Investmenttype column has repeated values of categories
#### categories have alphabetical error
this column has unformatted categories which results in repetition error(spaces between words)

here -Seed Funding and SeedFunding are same 




In [248]:
newdf['InvestmentType'].value_counts().index


In [249]:
newdf['InvestmentType']=newdf['InvestmentType'].apply(lambda x:x.replace(' ','').lower())

## Fomatting dates to time series

In [250]:
def rem_err_date(x):#function checks for error in format of date column in funding dataframe
    if re.search('.',x):
        return x.replace('.','')
    return x
newdf['Date']=newdf['Date'].apply(rem_err_date)
newdf['Date'].replace('12/052015','12/05/2015',inplace=True)
newdf['Date'].replace('15/012015','15/01/2015',inplace=True)
newdf['Date'].replace('22/01//2015','22/01/2015',inplace=True)
newdf['Date'].replace('13/042015','13/04/2015',inplace=True)
newdf['Date']=pd.to_datetime(newdf['Date'],format='%d/%m/%Y')

In [251]:
newdf.dtypes['Date']

## calculating number of investors for each startup
   - as various startups have multiple number of investors arrange according to the number of investors for a particular startup(startup name)

In [252]:
def calculate_n_investors(x):
    if  re.search(',',x) and x!='empty':
        return len(x.split(','))
    elif x!='empty':
        return 1
    else:
        return -1
newdf['numberofinvestors']=newdf['InvestorsName'].replace(np.NaN,'empty').apply(calculate_n_investors)

### most multiple investors for  individual startup

In [253]:
n_inv=newdf.loc[:,['StartupName','InvestorsName','numberofinvestors']].sort_values('numberofinvestors',ascending=False)

### visualizing the number of investors per startup


* visualizing the top 10 as per number of investors
* here maximum number of investors per startup is 10


In [254]:
n_inv.head(10)

- single funded startups are most
- there are least number of startups having 9 investors

In [255]:
plt.figure(figsize=(8,5))
sns.countplot(x='numberofinvestors',data=n_inv)


### FILLING MISSING VALUES IN  AmountInUSD

AmountinUSD column is formatted to integer 

In [256]:
def convert_AmountInUSD(x):
    if re.search(',',x):
        return (x.replace(',',''))
    return x
newdf['AmountInUSD']=newdf[newdf['AmountInUSD'].notnull()]['AmountInUSD'].apply(convert_AmountInUSD).astype('int')

In [257]:
newdf['AmountInUSD']=round(newdf['AmountInUSD'].fillna(np.mean(newdf['AmountInUSD'])))
newdf['AmountInUSD']=newdf['AmountInUSD'].astype('int')

In [258]:
sns.jointplot(x='numberofinvestors',y='AmountInUSD',data=newdf)

### FILLING MISSING VALUES IN  InvestmentType

In [259]:
newdf['InvestmentType'].fillna(method='bfill',inplace=True)#backward filling of null values

### Analysis of data after cleaning
 - here yellow bars represent the Missing data(NULL values)
 

In [260]:
plt.figure(figsize=(10,5))
sns.heatmap(newdf.isnull(),cmap='viridis',yticklabels=False,cbar=False)


# Indepth Analytics


In [261]:
newdf.head()

### Startups are only be initiate success by funding.
### Here we can see that best way to classify is to check the amount of funding and 
### the City Date is a time series which can show desired insights according to frequency

- funding of startups are highly dispersed and shown as investors are highly specific about startups


In [262]:
show(newdf.describe()['AmountInUSD'].astype(int))



how many unique startups are introduced in year between 2015-2017?
- 2001 startups

In [263]:
newdf['StartupName'].nunique()

# top 10 startups had most funding
paytm and flipkart were on the top of the run


In [264]:
tp10fund=show(newdf.groupby('StartupName')['AmountInUSD'].sum().sort_values(ascending=False))
tp10fund.head(10)



## WordCloud of best startups and sectors by amount of funding
 

In [265]:
from wordcloud import WordCloud, STOPWORDS


st=pd.Series(newdf.groupby('StartupName').sum()['AmountInUSD'].sort_values(ascending=False).head(40).index).head(30)
fig,ax=plt.subplots(nrows=2,ncols=1,figsize=(20,20),dpi=300)

for char in range(len(st)):
    st[char]=st[char].replace(' ','_')

wc=WordCloud(max_words=20,stopwords=set(st),background_color='darkgrey',random_state=0).generate(str(st.values[:30]))
ax[0].axis('off')
ax[0].imshow(wc)




c=np.array(newdf['IndustryVertical'].value_counts().head(30).index.unique())
for char in range(len(c)):
    c[char]=c[char].replace(' ','_')

               
wc=WordCloud(max_words=30,stopwords=set(c),background_color='plum',random_state=1).generate(str(c))

ax[1].axis('off')


ax[1].imshow(wc)
plt.rcParams['axes.facecolor'] = 'white'





#### DETAILS OF TOP 10 STARTUPS AS PER THE FUND GENERATED
   - below is the details about the top 10 startups on different Dates

In [266]:
#details of top 10 funded startups
def find(x):
    if x in tp10fund.head(10).index:
        return True
    return False

n=newdf[newdf['StartupName'].apply(find)]


 # which kind of investment did the top10 investors got
 
 - The top 10 investments were pivate equtiy and hence seed funding is nill
 
AS THE TOP 10 FUNDNDING AMOUNT ARE RECEIVED THROUGH PRIVATE EQUITY SO SORTING THE TOP 10 STARTUPS AS PER PRIVATE EQUITY
BELOW THE PRIVATE EQUITY COLUMN REPRESENTS THE COUNT OF INVESTMENT TYPE

In [267]:
pd.crosstab(n['StartupName'],columns=n['InvestmentType']).sort_values(by='privateequity',ascending=False)

## Insights regarding the best startup in terms of funding amount 

In [268]:
newdf[newdf['StartupName']=='Paytm']

### companies with most number of investors
here we can see that companies which have more number of investors are not proportionate to funding amount

In [269]:

cmi=show(newdf.groupby('StartupName')['numberofinvestors'].count().sort_values(ascending=False))
cmi.head(10)

#### below are the insights for number of inverstors to a particular start up 

In [270]:
fig=plt.figure(figsize=(10,5))
sns.barplot(y='numberofinvestors',x='StartupName',data=cmi.reset_index().head())
plt.show()

#### Here we can see that kernel density of startups having two and three number of investors is high 

In [271]:
sns.kdeplot(data=cmi.reset_index()['numberofinvestors'],gridsize=50,)#most are  2 or 3 in number
plt.title('Density estimation for number of investors ')
plt.show()

### Is funding depends upon number of investors
- here we can see that there is no relation between funding amount and number of investors

In [272]:
top10=tp10fund.join(cmi)
sns.heatmap(top10.corr(),annot=True)

### Investors  

In [273]:
investor=newdf[newdf['InvestorsName'].notnull()]['InvestorsName']
arr=np.array([])
for i in investor:
    for j in i.split(','):
        arr=np.append(arr,j.lower().strip())
        


### what are the total number of unique investors

In [274]:

show(arr)[0].nunique()

In [275]:
#investors with most funding
sh=show(arr)[0].unique()

### investors with most funding frequency

In [276]:
cinvestors=show(arr)[0].value_counts()[1:]
cinvestors.head(10)

In [277]:
print("Top Investors ")
plt.figure(figsize = (12,5))
bar= sns.barplot(x=cinvestors.index[:20],y=cinvestors.values[:20])
bar.set_xticklabels(bar.get_xticklabels(),rotation=70)
bar.set_title("Investors funding rate", fontsize=15)
bar.set_xlabel("", fontsize=12)
bar.set_ylabel("Count", fontsize=12)
plt.show()

In [278]:
Investors=cinvestors.index
arr =np.array(Investors)

        
for investor in range(len(arr)):
    arr[investor]=arr[investor].strip()
    arr[investor]=arr[investor].replace(' ','_')
    arr[investor]=arr[investor].replace("'",'')
    arr[investor]=arr[investor].replace("",'')


     


Ind=show(arr)[0].apply(lambda x: x.strip().lower())
Ind=Ind.values
fig=plt.figure(figsize=(10,10),dpi=700)

wc=WordCloud(max_words=30,stopwords=set(Ind),background_color='hotpink',random_state=1).generate(str(Ind[:30]))
plt.axis('off')
plt.imshow(wc)




### Top 10 Investors with higher funding amount

In [279]:
show(newdf.groupby('InvestorsName').sum()['AmountInUSD'].sort_values(ascending=False)).head(10)#top investors



### paring of amount and  number of investors :
- here we can see that most funding is generated through privateequity ( in terms of amount )
- most of the startups have seed funding (in terms of funding frequency )

### Total amount of funding recieved as per investment type 

In [280]:
newdf.groupby('InvestmentType').sum()['AmountInUSD']


### Total number of funding recieved as per investment type 

In [281]:
newdf['InvestmentType'].value_counts()



In [282]:
sns.pairplot(newdf,hue='InvestmentType',size=5)

# Top 10 industry sectors with most funding


### visualization of the growth rate of each sector (industry vertical)
- Here we can see that consumer internet sector has more funding (as per amount)
- here we can see that consumer internet sector has most frequency
- it also seen that debt funding and crowd funding are negligible as compared to other Investment types


In [283]:
# converting all industry vertical entries to lower to avoid category duplication
newdf['IndustryVertical']=newdf['IndustryVertical'].apply(lambda x:x.lower())


In [284]:
#in which sector there are most startups
d=newdf['IndustryVertical'].value_counts().head(5)
f=newdf.groupby('InvestmentType').sum()['AmountInUSD']
fig,ax=plt.subplots(nrows=1,ncols=2,figsize=(20,10))
labels=[d.index,f.index]
size=[d.values,f.values]
colors = [['gold', 'yellowgreen', 'lightcoral', 'lightskyblue','pink'],['green','pink','red','yellow']]
plt.axis('equal')
explode = ((0.1, 0, 0, 0,0),(-0.5,0.5,0.1,0.1))
ax[0].pie(size[0],explode=explode[0], labels=labels[0], colors=colors[0],
        autopct='%1.1f%%', shadow=True, startangle=140)

ax[1].pie(size[1],explode=explode[1], labels=labels[1], colors=colors[1],autopct='%1.5f%%', shadow=True, startangle=140)
plt.show()

In [285]:
plt.figure(figsize=(14,5))
iv=newdf['IndustryVertical'].value_counts().head(10)
iv.plot.bar()

plt.title('Frequency Startups Domain')
plt.show()

# Most preferrable cities as per Investment on startups
- insights provided shows that Banglore has the most average funding Amount


In [286]:
plt.figure(figsize=(17,12))
mean_amount = newdf.groupby('CityLocation').mean()["AmountInUSD"].astype('int').sort_values(ascending=False).iloc[1:].head(10)
squarify.plot(sizes=mean_amount.values,label=mean_amount.index, value=mean_amount.values,color=['crimson','seagreen','olive','hotpink','deepskyblue','grey','purple','lime','yellow','orange'])
plt.title('Distribution of Startups across Top cities')

###  here we can see that Ahmedabad is a Market place for dept-funding


In [287]:
fig=plt.figure(figsize=(15,50))

sns.barplot(y='CityLocation',x='AmountInUSD',data=newdf[newdf['CityLocation']!='india'],estimator=np.mean,hue='InvestmentType',palette='coolwarm')

#average investment in banglore is most
plt.show()
#amehdabad is the market place for dept funding


## Time Series Analysis

In [288]:
dnewdf=newdf.set_index('Date')
dnewdf.head()

### total funding in 2017 



In [289]:
dnewdf['2017']['AmountInUSD'].sum()

### total funding in 2016

In [290]:

dnewdf['2016']['AmountInUSD'].sum()


### total funding in 2015

In [291]:


dnewdf['2015']['AmountInUSD'].sum()

### total funding was most in 2015 and decreased

In [292]:

plt.title('total funding amount')
dnewdf.resample('AS')['AmountInUSD'].sum().plot.bar()


### Plotting Average Year-Start-wise graph for investment in period between January-2015 to Jan-2017 (Annual Starting)



In [293]:
q=dnewdf['AmountInUSD'].resample('AS').mean()
q.plot(kind='bar')
plt.title('average funding amount')


In [294]:
a=dnewdf['AmountInUSD'].resample('AS').sum()
fig,ax=plt.subplots(nrows=1,ncols=2)

fig.tight_layout(pad=3) # Or equivalently,  "plt.tight_layout()"
fig.set_figheight(11)
fig.set_figwidth(24)

explode = (0.1, 0.1, 0.1)
ax[0].pie(a,autopct='%1.1f%%',shadow=True,startangle=180,explode=explode,colors=['red','blue','green'],labels=['2015','2016','2017'])
ax[0].set_title('total funding amount per annum')

q.plot(color='y',ax=ax[1])
ax[1].set_title('average funding variation')
plt.show()


### Total variation of funding amount per month in period of 2015-2017

In [295]:
newdf['year']=newdf['Date'].dt.year
newdf['month']=newdf['Date'].dt.month

In [296]:
fig =plt.figure(figsize=(20,7))
fig.set_figheight
ts_month = newdf.groupby(['year', 'month']).agg({'AmountInUSD':'sum'})['AmountInUSD']
ts_month.plot(linewidth=4, color='crimson',marker="o", markersize=10, markerfacecolor='olive')
plt.ylabel('USD in Billions')
plt.xlabel('Year-Month');
plt.title('Funding Variation Per Month')


### Insights of 2015
- below we can visualize that total funding reached to its peak between the months of june - july and Aug - Sep
- End week of july was most attracted to investments
- Quarter 3 was seen as the most funded quarter of 2015

In [297]:
sns.set_context('poster',font_scale=1)


plt.figure(figsize=(20,6))


a=dnewdf['2015'].resample('MS').sum()['AmountInUSD'].plot()
a.set_title('total funding in 2015')
#insights of june-july and augest of 2015

fig2,axes = plt.subplots(nrows=1,ncols=3,figsize=(20,5))
a1=dnewdf['2015-06':'2015-8'].resample('d')['AmountInUSD'].sum().plot(ax=axes[0],lw=1)
a1.set_title('june-july-augest')

a2=sns.barplot(data=dnewdf['2015'].resample('Q').sum().reset_index(),y='AmountInUSD',x=['Q1','Q2','Q3','Q4'],ax=axes[1])
a2.set_title('Quaterly')
a3=dnewdf['2015'].resample('B')['AmountInUSD'].sum().plot(ax=axes[2],lw=1)
a3.set_title('funding by business days')



plt.show()

###  Insights of 2016

### it seems to be january and  augest was the most funding month in  2016

- A sudden increase is witnessed in the month of july - August

- june seems to be the decline in funding may be  due to goverment administration

- further after september  uniform low funding due to demonetization of indian currency

- early weeeks of augest seems to be the most funding time

- An inverse relation among funding and Quarter 


In [298]:
sns.set_context('poster',font_scale=1)


plt.figure(figsize=(22,6))


a=dnewdf['2016'].resample('MS').sum()['AmountInUSD'].plot()
a.set_title('total funding in 2016')
#insights of june-july and augest of 2015

fig2,axes = plt.subplots(nrows=1,ncols=3,figsize=(20,5))

a1=dnewdf['2016-06':'2016-10'].resample('d')['AmountInUSD'].sum().fillna(method='ffill').plot(ax=axes[0],lw=1)
a1.set_title('june-july-augest-sept')

sns.barplot(data=dnewdf['2016'].resample('Q').sum().reset_index(),y='AmountInUSD',x=['Q1','Q2','Q3','Q4'],ax=axes[1])

a3=dnewdf['2016'].resample('B')['AmountInUSD'].sum().plot(ax=axes[2],lw=1)
a3.set_title('funding by business days')




Insights of 2017
- Month of March and second Week of May had seen peak funding scenarios
- End ofMarch and mid of May witnessed the highest funding
- Quarter 2 has most funding  

In [299]:
sns.set_context('poster',font_scale=1)


plt.figure(figsize=(20,6))


a=dnewdf['2017'].resample('MS').sum()['AmountInUSD'].plot()
a.set_title('total funding in 2017')
#insights of june-july and augest of 2015

fig2,axes = plt.subplots(nrows=1,ncols=3,figsize=(20,5))
a1=dnewdf['2017-02':'2017-6'].resample('d')['AmountInUSD'].sum().fillna(method='ffill').plot(ax=axes[0],lw=1)
a1.set_title('funding by days')

a2=sns.barplot(data=dnewdf['2017'].resample('Q').sum().reset_index(),y='AmountInUSD',x=['Q1','Q2','Q3'],ax=axes[1])
a2.set_title('funding by quarter')

a3=dnewdf['2017'].resample('B')['AmountInUSD'].sum().plot(ax=axes[2],lw=1)
a3.set_title('funding by business days')


###  quartely  amount of investment

-  Quaterly funding is found to be normally distributed
-  Funding is highest in 3rd Quarter of 2015 

In [300]:
fig,ax=plt.subplots(1,2,figsize=(20,5))

sns.set_context('paper',font_scale=2)
dft=dnewdf.resample('Q').sum()
dft.index=dft.reset_index()['Date'].apply(lambda x: x.date())
a=sns.barplot(data=dft.reset_index(),x='Date',y='AmountInUSD',ax=ax[0])

plt.sca(ax[0])
plt.xticks(rotation=90)
plt.title('')

sns.distplot(dnewdf.resample('Q').sum()['AmountInUSD'],ax=ax[1])
dnewdf.resample('Q').sum()['AmountInUSD'].sort_index()



### Quaterly variation in Funding Amount

In [301]:

fig,ax=plt.subplots(nrows=2,ncols=1)
fig.set_figheight(13)
fig.set_figwidth(15)
ax[0].set_ylabel('USD in Billions')
quarter=dnewdf['AmountInUSD'].resample('Q').mean().astype('int')
quarter[1:].plot(linewidth=4, color='hotpink', marker="o", markersize=10, markerfacecolor='lime',ax=ax[0])
fig.tight_layout(pad=3) # Or equivalently,  "plt.tight_layout()"
plt.title('quartly funding amount')
explodes = (0,0.1, 0., 0,0,0,0,0.1,0.1,0.1,0.1)
ax[1].pie(quarter[0:],autopct='%1.1f%%',startangle=240,explode=explodes,labels=['2015-1','2015-2','2015-3','2015-4','2016-1','2016-2','2016-3','2016-4','2017-1','2017-2','2017-3'])

plt.show()


In [302]:
quarter

- In 2015 and 2016 the average funding was in the starting and mid quarters
- but in 2017 the average funding changed to mid to last quarters

- the pattern of average funding in 2015 vs 2017 is totally opposite

In [303]:
fig,axes = plt.subplots(nrows=1,ncols=3,figsize=(20,5))

plt.sca(axes[0])
plt.title('2015')
sns.kdeplot(dnewdf.resample('q').mean()['AmountInUSD']['2015'],ax=axes[0])

plt.sca(axes[1])
plt.title('2016')
sns.kdeplot(dnewdf.resample('q').mean()['AmountInUSD']['2016'],ax=axes[1])

plt.sca(axes[2])
plt.title('2017')
sns.kdeplot(dnewdf.resample('q').mean()['AmountInUSD']['2017'],ax=axes[2])


 Private equity is seen to be most favourable Investment Mode for high funding amount per startup

In [304]:
plt.scatter(x=newdf['AmountInUSD'],y=newdf['InvestmentType'])
plt.title('InvestmentType Vs AmountInUSD')

### Investment type in 2015



- private equity is more dispersed according to amount funded  
 and more amount is invested through private equity per startup on the otherhand
 seed funding is less dispersed according to amount funded and low amount is funded using this but 
 frequency of seed funding is more as compared to private equity
- No dept funding occured in 2015

In [305]:
plt.figure(figsize=(15,8))
d2015=dnewdf['2015']
print(sns.stripplot(data=d2015,x='InvestmentType',y='AmountInUSD',jitter=True,hue='numberofinvestors'))

print(d2015['InvestmentType'].value_counts())
d2015[d2015['InvestmentType']=='privateequity'].sort_values(by='AmountInUSD',ascending=False).head(5)

- private equity is more dispersed according to amount funded
 and more amount is invested through private equity per startup on the otherhand seed funding is less dispersed         according to amount funded and low amount is funded using this but frequency of seed funding is more as compared 
 to private equity
- As compared to 2015v seed funding has slightly more deviated 
- No dept funding and crowd funding occured in 2016

- snapdeal and makemytrip was funded most via private equity 

In [306]:
plt.figure(figsize=(15,8))
d2016=dnewdf['2016']
print(sns.stripplot(data=d2016,x='InvestmentType',y='AmountInUSD',jitter=True,hue='numberofinvestors'))

print(d2016['InvestmentType'].value_counts())
d2016[d2016['InvestmentType']=='privateequity'].sort_values(by='AmountInUSD',ascending=False).head(2)

### InvestmentType In 2017
- Private Equity is not so dispersed as compared to previous two years
- Debt funding slightly taken place

flipkart and paytm had max funding at once  and uncommon to others

In [307]:
plt.figure(figsize=(15,8))
d2017=dnewdf['2017']
print(sns.stripplot(data=d2017,x='InvestmentType',y='AmountInUSD',jitter=True,hue='numberofinvestors'))

print(d2017['InvestmentType'].value_counts())
d2017[d2017['InvestmentType']=='privateequity'].sort_values(by='AmountInUSD',ascending=False).head(2)

## startups with funding on multiple days
- Startups having 2 number of Investors are higher


In [308]:
fig,ax=plt.subplots(nrows=1,ncols=2)
fig.set_figheight(5)
fig.set_figwidth(20)

df_v=dnewdf['StartupName'].value_counts()
df_v[df_v>1].hist(ax=ax[0])
df_v[df_v>1].head(10)
ax[0].set_title('frequency of count of investors per startup ')
ax[0].set_ylabel('frequency')
ax[0].set_xlabel('number of investors')
ax[1].set_title('startups most number of times invested')
ax[1].set_ylabel('frequency')

newdf['StartupName'].value_counts().head(10).plot(kind='bar')

plt.figure(figsize=(15,12))
newdf['StartupName'].value_counts().head(10).plot(kind='pie',colormap='coolwarm')
plt.title("startups most number of times invested", fontsize=16)
plt.show()

## STORY TELLING - CONCLUSION


1.funding startups are highly dispersed as investors are highly specific about choosing startups <br>
2.there were more than 2000 new startups funded in the year between 2015-2017<br>
3.Paytm and flipkart were funded most <br>
4.top 10 investments are made through private equity<br>
5.paytm was mostly funded on different days by single investor<br>
6.swiggy was invested by most number of investors<br>
7.as per startups 2 and 3 were most frequent combination of investors<br>
8.there were no relation between funded amount and number of investors per startup<br>
9.there were more than 2000 unique investors<br>
10.sequoia capital invested in most number of startups<br>
11.soft bank group invested maximum amount of money<br>
12.most funding came through private equity<br>
13.Crowd funding and dept funding were less preferred by the investors<br>
14.seed funding was less dispersed but private equity is choosen to made large investments per startup<br>
15.consumer internet sector got most ammount of funding and attracted the most investors<br>
16.Banglore had the most average funding <br>
17.Ahmedabad seems to be an anomalie as it only witnessed the debt funding<br>
18.Maximum Total  funding was generated  in the  year 2015 and then it slowly decreased with increase in years<br>
19.average funding was most in 2017<br>
20.In 2015 the period of june-oct was funded most and quarter 3 was funded most<br>
21.In 2016 period of june and september were most funded and quterly funding decreased with increase in time<br>
22.In 2017 the period between march- may had recieved most fundings<br>
23.3rd quarter  of 2015 was invested most and funding amount was normally distributed<br>
24.starting months of 2015,mid of 2016 and 3rd Quarter of 2017 were funded most as per individual investment<br>
25.2016 had highest amount of investments<br>
26.In 2015 there was slight fund generated through crowd funding<br>
27.In 2016 there was slight variation in seed funding by amount<br>
28.In 2017 there dept funding witnessed for first time but in less fraction<br>
 